## Multi-Label News Article Classification

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
import wandb
import os
from collections import Counter
wandb.login(key="cb255956b462ea96bcedb0051bc14581a02f25c3")
from sklearn.preprocessing import StandardScaler


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Admin\_netrc
wandb: Currently logged in as: ketakishetye0619 (ketakishetye0619-iiit-hyderabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


### Data Preprocessing

In [3]:
df_train = pd.read_csv("news-article/train.csv")
print(df_train.isnull().sum())
df_train.dropna(inplace=True)
df_train['category'] = df_train['category'].apply(lambda x: x.split(','))

df_test = pd.read_csv("news-article/test.csv")
print(df_test.isnull().sum())
df_test.dropna(inplace=True)
df_test['category'] = df_test['category'].apply(lambda x: x.split(','))


document    0
category    0
dtype: int64
document    0
category    0
dtype: int64


In [4]:
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(df_train['category'])
print(y_train.shape)
y_test = mlb.transform(df_test['category'])
print(y_test.shape)
print("Categories:", mlb.classes_)

(7769, 90)
(3019, 90)
Categories: ['acq' 'alum' 'barley' 'bop' 'carcass' 'castor-oil' 'cocoa' 'coconut'
 'coconut-oil' 'coffee' 'copper' 'copra-cake' 'corn' 'cotton' 'cotton-oil'
 'cpi' 'cpu' 'crude' 'dfl' 'dlr' 'dmk' 'earn' 'fuel' 'gas' 'gnp' 'gold'
 'grain' 'groundnut' 'groundnut-oil' 'heat' 'hog' 'housing' 'income'
 'instal-debt' 'interest' 'ipi' 'iron-steel' 'jet' 'jobs' 'l-cattle'
 'lead' 'lei' 'lin-oil' 'livestock' 'lumber' 'meal-feed' 'money-fx'
 'money-supply' 'naphtha' 'nat-gas' 'nickel' 'nkr' 'nzdlr' 'oat' 'oilseed'
 'orange' 'palladium' 'palm-oil' 'palmkernel' 'pet-chem' 'platinum'
 'potato' 'propane' 'rand' 'rape-oil' 'rapeseed' 'reserves' 'retail'
 'rice' 'rubber' 'rye' 'ship' 'silver' 'sorghum' 'soy-meal' 'soy-oil'
 'soybean' 'strategic-metal' 'sugar' 'sun-meal' 'sun-oil' 'sunseed' 'tea'
 'tin' 'trade' 'veg-oil' 'wheat' 'wpi' 'yen' 'zinc']


In [5]:
def tokenize(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    return text.split()

df_train['tokens'] = df_train['document'].apply(tokenize)
df_test['tokens'] = df_test['document'].apply(tokenize)

print(df_train['tokens'].shape)

(7769,)


In [6]:
doc_freq = Counter()
for doc in df_train['tokens']:
    unique_words = set(doc)
    doc_freq.update(unique_words)
    
print(len(doc_freq))

num_docs = len(df_train)
idf_train = {word: np.log(num_docs / (1 + doc_freq[word])) for word in doc_freq.keys()}
print(num_docs)

feature_words = sorted(doc_freq.keys(), key=lambda w: doc_freq[w], reverse=True)[:5000]
feature_indices = {word: i for i, word in enumerate(feature_words)}

x_train = np.zeros((len(df_train), len(feature_words)))
x_test = np.zeros((len(df_test), len(feature_words)))

for i, tokens in enumerate(df_train['tokens']):
    if len(tokens) == 0:
        continue
    
    tf = Counter(tokens)
    doc_len = len(tokens)
    
    for word in set(tokens).intersection(feature_words):
        idx = feature_indices[word]
        tf_val = tf[word] / doc_len
        idf_val = idf_train[word]
        x_train[i, idx] = tf_val * idf_val

for i, tokens in enumerate(df_test['tokens']):
    if len(tokens) == 0:
        continue
    
    tf = Counter(tokens)
    doc_len = len(tokens)
    
    for word in set(tokens).intersection(feature_words):
        idx = feature_indices[word]
        tf_val = tf[word] / doc_len
        idf_val = idf_train[word]
        x_test[i, idx] = tf_val * idf_val

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print(f"Train: {x_train.shape}, {y_train.shape}")
print(f"Validation: {x_val.shape}, {y_val.shape}")
print(f"Test: {x_test.shape}, {y_test.shape}")
print(y_train[0])

41198
7769
Train: (6215, 5000), (6215, 90)
Validation: (1554, 5000), (1554, 90)
Test: (3019, 5000), (3019, 90)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [7]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

### MLP MultiLabel

In [8]:
class MLP():
    def __init__(self,input_dim,hidden_dim,output_dim,activation):
        self.in_hidden_weights=np.random.randn(input_dim, hidden_dim)*np.sqrt(2/input_dim)
        self.out_hidden_weights=np.random.randn(hidden_dim,output_dim)*np.sqrt(2/hidden_dim)
        self.bias_hidden = np.zeros((1, hidden_dim))
        self.bias_output=np.zeros((1,output_dim))
        self.activation=activation
        
    def apply_activation(self,x):
        if self.activation=='relu':
            activation = np.maximum(0, x)  
        elif self.activation=='sigmoid':
            activation = 1 / (1 + np.exp(-x))
        elif self.activation=='tanh':
            activation = np.tanh(x)  
            
        return activation
    
    def apply_derivative(self,x):
        if self.activation=='relu':
            derivative=np.where(x > 0, 1, 0)
        elif self.activation=='sigmoid':
            activation = 1 / (1 + np.exp(-x))
            derivative= activation * (1 - activation)  
        elif self.activation=='tanh':
            activation = np.tanh(x)  
            derivative = 1 - activation ** 2  
            
        return derivative
    
    def sigmoid(self,x):
        return 1 / (1 + np.exp(-x))
    
    def forward(self,x):
        self.hidden_layer_input=np.dot(x,self.in_hidden_weights)+self.bias_hidden
        self.hidden_layer_output=self.apply_activation(self.hidden_layer_input)
        self.final_input=np.dot(self.hidden_layer_output,self.out_hidden_weights)+self.bias_output
        self.final_output=self.sigmoid(self.final_input)
        return self.final_output
    
    def backward_propogation(self,x,y_true,y_pred,learning_rate):
        batch_size=x.shape[0]
        output_layer_error=(y_pred-y_true)/batch_size
        hidden_layer_error=(np.dot(output_layer_error,self.out_hidden_weights.T))*self.apply_derivative(self.hidden_layer_input) # total contribution of each hidden neuron to the overall output error and multiplied by derivative as scales the hidden error based on how much each hidden neuron is "activated."

        self.out_hidden_weights=self.out_hidden_weights-learning_rate*(np.dot(self.hidden_layer_output.T,output_layer_error))
        self.bias_output=self.bias_output-learning_rate*(np.sum(output_layer_error,axis=0,keepdims=True))
        
        self.in_hidden_weights=self.in_hidden_weights-learning_rate*(np.dot(x.T,hidden_layer_error))
        self.bias_hidden=self.bias_hidden-learning_rate*(np.sum(hidden_layer_error,axis=0,keepdims=True))

    def binary_cross_entropy(self,y_true, y_pred, epsilon=1e-12):
        loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
        return loss
        
    def calculate_accuracy(self, y_true, y_pred,threshold=0.5):
        y_pred_binary = (y_pred >= threshold).astype(int)
        y_pred_binary = (y_pred >= threshold).astype(int)
        exact_match = np.mean(np.all(y_pred_binary == y_true, axis=1))
        hamming_accuracy = np.mean(y_pred_binary == y_true)
        return exact_match*100,hamming_accuracy*100
        
    def predict(self, x):
        return self.forward(x)
    
    def train(self,x_train,y_true,num_epochs,learning_rate,x_test,y_test,batch_size):
        
        total_num_train_samples=x_train.shape[0]
        if batch_size is None:
            batch_size=total_num_train_samples

        for epoch in tqdm(range(num_epochs), desc="Training Progress"):
            
            indices=np.random.permutation(total_num_train_samples)
            x_shuffled_train = x_train[indices]
            y_shuffled_train= y_true[indices]
            
            epoch_loss = 0
            epoch_accuracy = 0
            num_batches = 0
            epoch_hamming_accuracy = 0
        
            for i in range(0, total_num_train_samples, batch_size):
                end = min(i + batch_size, total_num_train_samples)
                x_batch = x_shuffled_train[i:end]
                y_batch = y_shuffled_train[i:end]
                
                y_pred=self.forward(x_batch)
                # print(y_pred.shape)
                # print(y_train.shape)
                self.backward_propogation(x_batch,y_batch,y_pred,learning_rate)
                
                loss=self.binary_cross_entropy(y_batch,y_pred)
                epoch_loss=epoch_loss+loss
                
                accuracy,hamming_accuracy=self.calculate_accuracy(y_batch,y_pred)
                epoch_accuracy=epoch_accuracy+accuracy
                epoch_hamming_accuracy=epoch_hamming_accuracy+hamming_accuracy
                
                num_batches=num_batches+1
                
            y_pred_val=self.predict(x_val)
            val_loss=self.binary_cross_entropy(y_val,y_pred_val)
            val_accuracy,val_hamming_accuracy=self.calculate_accuracy(y_val,y_pred_val)
            
            wandb.log({"Epoch": epoch, "Train Loss": epoch_loss/num_batches, "Validation Loss": val_loss, "Validation Accuracy": val_accuracy, "Train Accuracy":epoch_accuracy/num_batches})
            
            if (epoch+1)%10==0:
                print(f"Epoch: {epoch},Train Loss:{epoch_loss/num_batches},Train Accuracy:{epoch_accuracy/num_batches},Train Hamming Accuracy:{epoch_hamming_accuracy/num_batches}")
                print(f"Epoch: {epoch},Validation Loss:{val_loss},Validation Accuracy:{val_accuracy},Validation Hamming Accuracy:{val_hamming_accuracy}")
            elif epoch==0:
                print(f"Epoch: {epoch},Train Loss:{epoch_loss/num_batches},Train Accuracy:{epoch_accuracy/num_batches},Train Hamming Accuracy:{epoch_hamming_accuracy/num_batches}")
                print(f"Epoch: {epoch},Validation Loss:{val_loss},Validation Accuracy:{val_accuracy},Validation Hamming Accuracy:{val_hamming_accuracy}")
            


### Relu-BGD

In [67]:
np.random.seed(42)
input_dim=5000
hidden_dim=1024
learning_rate=0.01
num_epochs=100
activation='relu'
batch_size=None
output_dim=y_train.shape[1]

In [68]:
wandb.init(project="SMAI-A2", name="2.3-Relu-BGD")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   1%|          | 1/100 [00:01<02:34,  1.57s/it]

Epoch: 0,Train Loss:0.9167808637657126,Train Accuracy:0.0,Train Hamming Accuracy:48.58085277554304
Epoch: 0,Validation Loss:0.35498360531814777,Validation Accuracy:0.0,Validation Hamming Accuracy:87.81781781781783


Training Progress:  10%|█         | 10/100 [00:18<02:53,  1.92s/it]

Epoch: 9,Train Loss:0.10121757961011343,Train Accuracy:17.248592115848755,Train Hamming Accuracy:98.65987306695271
Epoch: 9,Validation Loss:0.09609773184829741,Validation Accuracy:17.63191763191763,Validation Hamming Accuracy:98.65079365079366


...

Epoch: 89,Train Loss:0.035872731658923886,Train Accuracy:51.00563153660499,Train Hamming Accuracy:99.22499329578976
Epoch: 89,Validation Loss:0.04560405732038002,Validation Accuracy:41.6988416988417,Validation Hamming Accuracy:99.07407407407408


Training Progress: 100%|██████████| 100/100 [02:53<00:00,  1.73s/it]

Epoch: 99,Train Loss:0.033732124359231785,Train Accuracy:53.193885760257444,Train Hamming Accuracy:99.25914007329936
Epoch: 99,Validation Loss:0.04400457323939395,Validation Accuracy:43.43629343629344,Validation Hamming Accuracy:99.1005291005291


Test Loss:0.04710621023254192,Test Accuracy:48.22789002981119,Test Hamming Accuracy:99.12333002097824


Epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█
Train Accuracy,▁▂▂▃▃▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇███████
Train Loss,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▂▂▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████
Validation Loss,█▅▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,99
Train Accuracy,53.19389
Train Loss,0.03373
Validation Accuracy,43.43629
Validation Loss,0.044


In [10]:
np.random.seed(42)
input_dim=5000
hidden_dim=512
learning_rate=0.01
num_epochs=100
activation='relu'
batch_size=None
output_dim=y_train.shape[1]

In [11]:
wandb.init(project="SMAI-A2", name="2.3-Relu-BGD-Hyperparam1")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   1%|          | 1/100 [00:00<01:22,  1.20it/s]

Epoch: 0,Train Loss:0.8763924419307746,Train Accuracy:0.0,Train Hamming Accuracy:51.18101367658889
Epoch: 0,Validation Loss:0.5817003507954865,Validation Accuracy:0.0,Validation Hamming Accuracy:72.21006721006721


Training Progress:  10%|█         | 10/100 [00:09<01:28,  1.02it/s]

Epoch: 9,Train Loss:0.146229666288946,Train Accuracy:12.952534191472246,Train Hamming Accuracy:98.2401001162063
Epoch: 9,Validation Loss:0.13227877746513064,Validation Accuracy:14.607464607464607,Validation Hamming Accuracy:98.32904332904333


...

Training Progress:  90%|█████████ | 90/100 [01:26<00:09,  1.02it/s]

Epoch: 89,Train Loss:0.03784552055517517,Train Accuracy:49.782783588093324,Train Hamming Accuracy:99.1967462232949
Epoch: 89,Validation Loss:0.047460830360257715,Validation Accuracy:40.99099099099099,Validation Hamming Accuracy:99.05762905762906


Training Progress: 100%|██████████| 100/100 [01:35<00:00,  1.05it/s]

Epoch: 99,Train Loss:0.03548004473798695,Train Accuracy:51.72968624296058,Train Hamming Accuracy:99.23160811656388
Epoch: 99,Validation Loss:0.04569883727435366,Validation Accuracy:42.02059202059202,Validation Hamming Accuracy:99.07335907335907
Test Loss:0.04825348012792756,Test Accuracy:45.21364690294799,Test Hamming Accuracy:99.07916528651872


Epoch,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇██
Train Accuracy,▁▁▁▂▂▃▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇██████
Train Loss,█▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▂▂▃▃▄▄▄▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████
Validation Loss,█▅▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,99
Train Accuracy,51.72969
Train Loss,0.03548
Validation Accuracy,42.02059
Validation Loss,0.0457


In [12]:
np.random.seed(42)
input_dim=5000
hidden_dim=512
learning_rate=0.001
num_epochs=100
activation='relu'
batch_size=None
output_dim=y_train.shape[1]

In [13]:
wandb.init(project="SMAI-A2", name="2.3-Relu-BGD-Hyperparam2")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]C:\Users\Admin\AppData\Local\Temp\ipykernel_28664\3730871249.py:53: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
Training Progress:   1%|          | 1/100 [00:00<01:23,  1.18it/s]

Epoch: 0,Train Loss:0.8763924419307746,Train Accuracy:0.0,Train Hamming Accuracy:51.18101367658889
Epoch: 0,Validation Loss:inf,Validation Accuracy:0.0,Validation Hamming Accuracy:53.24824824824825


Training Progress:  10%|█         | 10/100 [00:08<01:21,  1.11it/s]

Epoch: 9,Train Loss:0.6114914481557494,Train Accuracy:0.0,Train Hamming Accuracy:68.22275855904174
Epoch: 9,Validation Loss:0.6193649301621728,Validation Accuracy:0.0,Validation Hamming Accuracy:69.75475475475476


...

Training Progress:  90%|█████████ | 90/100 [01:21<00:09,  1.10it/s]

Epoch: 89,Train Loss:0.15584520063736215,Train Accuracy:11.472244569589703,Train Hamming Accuracy:98.07151157593636
Epoch: 89,Validation Loss:0.1491452924459062,Validation Accuracy:11.389961389961389,Validation Hamming Accuracy:98.01515801515801


Training Progress: 100%|██████████| 100/100 [01:30<00:00,  1.11it/s]

Epoch: 99,Train Loss:0.14470571701209078,Train Accuracy:14.062751407884152,Train Hamming Accuracy:98.26441405202468
Epoch: 99,Validation Loss:0.13901286760142248,Validation Accuracy:13.77091377091377,Validation Hamming Accuracy:98.22179322179322
Test Loss:0.1436336775141032,Test Accuracy:11.957601854918847,Test Hamming Accuracy:98.198078834051


Epoch,▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▆▆▆▆▆▆▆▇▇▇███
Train Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▃▃▄▄▄▄▄▅▅▅▆▆▆▇▇██
Train Loss,█▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▅▅▅▅▆▆▆▆▇▇▇█
Validation Loss,██▇▇▆▆▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,99
Train Accuracy,14.06275
Train Loss,0.14471
Validation Accuracy,13.77091
Validation Loss,0.13901


### Relu-SGD

In [80]:
np.random.seed(42)
input_dim=5000
hidden_dim=1024
learning_rate=0.01
num_epochs=100
activation='relu'
batch_size=None
output_dim=y_train.shape[1]

In [76]:
wandb.init(project="SMAI-A2", name="2.3-Relu-SGD")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   1%|          | 1/100 [00:01<02:29,  1.51s/it]

Epoch: 0,Train Loss:0.9167808637657126,Train Accuracy:0.0,Train Hamming Accuracy:48.58085277554304
Epoch: 0,Validation Loss:0.35498360531814777,Validation Accuracy:0.0,Validation Hamming Accuracy:87.81781781781783


Training Progress:  10%|█         | 10/100 [00:16<02:30,  1.68s/it]

Epoch: 9,Train Loss:0.10121757961011343,Train Accuracy:17.248592115848755,Train Hamming Accuracy:98.65987306695271
Epoch: 9,Validation Loss:0.09609773184829741,Validation Accuracy:17.63191763191763,Validation Hamming Accuracy:98.65079365079366


...

Epoch: 89,Train Loss:0.035872731658923886,Train Accuracy:51.00563153660499,Train Hamming Accuracy:99.22499329578976
Epoch: 89,Validation Loss:0.04560405732038002,Validation Accuracy:41.6988416988417,Validation Hamming Accuracy:99.07407407407408


Training Progress: 100%|██████████| 100/100 [02:49<00:00,  1.69s/it]

Epoch: 99,Train Loss:0.033732124359231785,Train Accuracy:53.193885760257444,Train Hamming Accuracy:99.25914007329936
Epoch: 99,Validation Loss:0.04400457323939395,Validation Accuracy:43.43629343629344,Validation Hamming Accuracy:99.1005291005291


Test Loss:0.04710621023254192,Test Accuracy:48.22789002981119,Test Hamming Accuracy:99.12333002097824


Epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█
Train Accuracy,▁▁▂▂▃▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████
Train Loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▂▂▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█████
Validation Loss,█▅▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,99
Train Accuracy,53.19389
Train Loss,0.03373
Validation Accuracy,43.43629
Validation Loss,0.044


In [30]:
np.random.seed(42)
input_dim=5000
hidden_dim=512
learning_rate=0.01
num_epochs=100
activation='relu'
batch_size=None
output_dim=y_train.shape[1]

In [31]:
wandb.init(project="SMAI-A2", name="2.3-Relu-SGD-Hyperparam1")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   1%|          | 1/100 [00:00<01:15,  1.31it/s]

Epoch: 0,Train Loss:0.8763924419307746,Train Accuracy:0.0,Train Hamming Accuracy:51.18101367658889
Epoch: 0,Validation Loss:0.5817003507954865,Validation Accuracy:0.0,Validation Hamming Accuracy:72.21006721006721


Training Progress:  10%|█         | 10/100 [00:09<01:23,  1.08it/s]

Epoch: 9,Train Loss:0.146229666288946,Train Accuracy:12.952534191472246,Train Hamming Accuracy:98.2401001162063
Epoch: 9,Validation Loss:0.13227877746513064,Validation Accuracy:14.607464607464607,Validation Hamming Accuracy:98.32904332904333


...

Training Progress:  90%|█████████ | 90/100 [01:21<00:08,  1.13it/s]

Epoch: 89,Train Loss:0.03784552055517517,Train Accuracy:49.782783588093324,Train Hamming Accuracy:99.1967462232949
Epoch: 89,Validation Loss:0.047460830360257715,Validation Accuracy:40.99099099099099,Validation Hamming Accuracy:99.05762905762906


Training Progress: 100%|██████████| 100/100 [01:31<00:00,  1.10it/s]

Epoch: 99,Train Loss:0.03548004473798695,Train Accuracy:51.72968624296058,Train Hamming Accuracy:99.23160811656388
Epoch: 99,Validation Loss:0.04569883727435366,Validation Accuracy:42.02059202059202,Validation Hamming Accuracy:99.07335907335907
Test Loss:0.04825348012792756,Test Accuracy:45.21364690294799,Test Hamming Accuracy:99.07916528651872


Epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
Train Accuracy,▅█▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▅▅▅▅▅
Train Loss,▁█▆▄▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,███▁▁▁▂▂▃▃▃▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▅▅▅
Validation Loss,▁▁█▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
Epoch,99
Train Accuracy,51.72969
Train Loss,0.03548
Validation Accuracy,42.02059
Validation Loss,0.0457


In [32]:
np.random.seed(42)
input_dim=5000
hidden_dim=1024
learning_rate=0.001
num_epochs=100
activation='relu'
batch_size=None
output_dim=y_train.shape[1]

In [33]:
wandb.init(project="SMAI-A2", name="2.3-Relu-SGD-Hyperparam2")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   1%|          | 1/100 [00:01<02:44,  1.66s/it]

Epoch: 0,Train Loss:0.9174037545397506,Train Accuracy:0.0,Train Hamming Accuracy:48.5207830517565
Epoch: 0,Validation Loss:0.9233020882333469,Validation Accuracy:0.0,Validation Hamming Accuracy:52.56542256542257


Training Progress:  10%|█         | 10/100 [00:16<02:26,  1.63s/it]

Epoch: 9,Train Loss:0.46933166703892454,Train Accuracy:0.0,Train Hamming Accuracy:79.3909001519621
Epoch: 9,Validation Loss:0.4513197080610383,Validation Accuracy:0.0,Validation Hamming Accuracy:81.17546117546117


...

Epoch: 89,Train Loss:0.10801722231970882,Train Accuracy:16.427996781979083,Train Hamming Accuracy:98.61839635290963
Epoch: 89,Validation Loss:0.10544731569110927,Validation Accuracy:15.444015444015443,Validation Hamming Accuracy:98.57071357071358


Training Progress: 100%|██████████| 100/100 [02:49<00:00,  1.69s/it]

Epoch: 99,Train Loss:0.10212061208726779,Train Accuracy:17.779565567176185,Train Hamming Accuracy:98.6580852775543
Epoch: 99,Validation Loss:0.100399552500795,Validation Accuracy:16.53796653796654,Validation Hamming Accuracy:98.6086086086086


Test Loss:0.10296203568012342,Test Accuracy:18.25107651540245,Test Hamming Accuracy:98.63089323175444


Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇██
Train Accuracy,▁▁▁▁▁▁▁▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇████
Train Loss,█▇▇▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▂▃▄▄▄▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇████
Validation Loss,█▇▆▆▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,99
Train Accuracy,17.77957
Train Loss,0.10212
Validation Accuracy,16.53797
Validation Loss,0.1004


### Relu-MBGD

In [77]:
np.random.seed(42)
input_dim=5000
hidden_dim=1024
learning_rate=0.01
num_epochs=50
activation='relu'
batch_size=256
output_dim=y_train.shape[1]

In [78]:
wandb.init(project="SMAI-A2", name="2.3-Relu-MBGD")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   2%|▏         | 1/50 [00:02<01:55,  2.35s/it]

Epoch: 0,Train Loss:0.14552341660441584,Train Accuracy:17.374339788732396,Train Hamming Accuracy:96.08194933489827
Epoch: 0,Validation Loss:0.0691333320816532,Validation Accuracy:27.220077220077222,Validation Hamming Accuracy:98.83669383669384


Training Progress:  20%|██        | 10/50 [00:23<01:32,  2.31s/it]

Epoch: 9,Train Loss:0.018142677865957225,Train Accuracy:71.68617957746478,Train Hamming Accuracy:99.55469483568075
Epoch: 9,Validation Loss:0.0321642925463274,Validation Accuracy:52.96010296010296,Validation Hamming Accuracy:99.26640926640925


...

Epoch: 39,Train Loss:0.0038407689796089483,Train Accuracy:96.39348591549296,Train Hamming Accuracy:99.94062010954616
Epoch: 39,Validation Loss:0.025304773801154545,Validation Accuracy:62.93436293436293,Validation Hamming Accuracy:99.4036894036894


Training Progress: 100%|██████████| 50/50 [01:52<00:00,  2.26s/it]

Epoch: 49,Train Loss:0.0029355810720688926,Train Accuracy:97.71875,Train Hamming Accuracy:99.9625
Epoch: 49,Validation Loss:0.02517176906836825,Validation Accuracy:63.64221364221364,Validation Hamming Accuracy:99.41226941226941


Test Loss:0.032840260546808275,Test Accuracy:61.70917522358397,Test Hamming Accuracy:99.35298664016783


Epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
Train Accuracy,▁▃▃▄▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇███████████████████
Train Loss,█▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▃▄▄▄▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████
Validation Loss,█▆▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,49
Train Accuracy,97.71875
Train Loss,0.00294
Validation Accuracy,63.64221
Validation Loss,0.02517


In [14]:
np.random.seed(42)
input_dim=5000
hidden_dim=512
learning_rate=0.01
num_epochs=50
activation='relu'
batch_size=256
output_dim=y_train.shape[1]

In [15]:
wandb.init(project="SMAI-A2", name="2.3-Relu-MBGD-Hyperparam1")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   2%|▏         | 1/50 [00:01<01:01,  1.25s/it]

Epoch: 0,Train Loss:0.194783658060067,Train Accuracy:14.967649647887324,Train Hamming Accuracy:94.40222026604067
Epoch: 0,Validation Loss:0.0790851290474612,Validation Accuracy:26.576576576576578,Validation Hamming Accuracy:98.80809380809382


Training Progress:  20%|██        | 10/50 [00:11<00:47,  1.20s/it]

Epoch: 9,Train Loss:0.018982077040500252,Train Accuracy:69.9168133802817,Train Hamming Accuracy:99.52573845852898
Epoch: 9,Validation Loss:0.03312911636581071,Validation Accuracy:52.96010296010296,Validation Hamming Accuracy:99.25210925210925


...

Training Progress:  80%|████████  | 40/50 [00:49<00:12,  1.25s/it]

Epoch: 39,Train Loss:0.004265609235760031,Train Accuracy:95.64964788732394,Train Hamming Accuracy:99.926621185446
Epoch: 39,Validation Loss:0.025096119006453373,Validation Accuracy:62.74131274131274,Validation Hamming Accuracy:99.4029744029744


Training Progress: 100%|██████████| 50/50 [01:01<00:00,  1.23s/it]

Epoch: 49,Train Loss:0.0032925371023508703,Train Accuracy:97.24053697183099,Train Hamming Accuracy:99.95086316510172
Epoch: 49,Validation Loss:0.024788902027683885,Validation Accuracy:63.77091377091377,Validation Hamming Accuracy:99.41512941512941
Test Loss:0.030967951015967353,Test Accuracy:62.47101689301093,Test Hamming Accuracy:99.3577711530676


Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
Train Accuracy,▁▃▃▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇███████████████████
Train Loss,█▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▃▃▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇██████████████████
Validation Loss,█▅▄▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,49
Train Accuracy,97.24054
Train Loss,0.00329
Validation Accuracy,63.77091
Validation Loss,0.02479


In [19]:
np.random.seed(42)
input_dim=5000
hidden_dim=1024
learning_rate=0.01
num_epochs=50
activation='relu'
batch_size=256
output_dim=y_train.shape[1]

In [20]:
wandb.init(project="SMAI-A2", name="2.3-Relu-MBGD-Hyperparam2")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   2%|▏         | 1/50 [00:02<01:55,  2.36s/it]

Epoch: 0,Train Loss:0.14569912514849215,Train Accuracy:17.165052816901408,Train Hamming Accuracy:96.07677767996869
Epoch: 0,Validation Loss:0.06911173240994845,Validation Accuracy:26.126126126126124,Validation Hamming Accuracy:98.82668382668382


Training Progress:  20%|██        | 10/50 [00:24<01:38,  2.46s/it]

Epoch: 9,Train Loss:0.01816870918346523,Train Accuracy:71.86751760563381,Train Hamming Accuracy:99.55809859154928
Epoch: 9,Validation Loss:0.032321083537668605,Validation Accuracy:53.153153153153156,Validation Hamming Accuracy:99.27355927355927


...

Epoch: 39,Train Loss:0.003839506648105762,Train Accuracy:96.42473591549296,Train Hamming Accuracy:99.94062010954616
Epoch: 39,Validation Loss:0.02552591376616162,Validation Accuracy:62.419562419562425,Validation Hamming Accuracy:99.4058344058344


Training Progress: 100%|██████████| 50/50 [02:00<00:00,  2.41s/it]

Epoch: 49,Train Loss:0.002932277664949726,Train Accuracy:97.71875,Train Hamming Accuracy:99.96232638888888
Epoch: 49,Validation Loss:0.025405535079844634,Validation Accuracy:63.44916344916345,Validation Hamming Accuracy:99.41798941798942


Test Loss:0.03314635141496776,Test Accuracy:61.11295130838026,Test Hamming Accuracy:99.34709800890656


Epoch,▁▁▂▂▃▄▄▄▄▅▆▆▆▆▇▇█▁▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇▇██
Train Accuracy,▁▂▂▄▄▅▆▆▆▇▇▇▇███████▂▄▅▆▆▆▇▇▇▇▇█████████
Train Loss,█▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▃▃▅▅▅▆▆▆▇▇▇▇▇███████▁▂▄▄▅▆▆▆▇▇▇▇▇███████
Validation Loss,█▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,49
Train Accuracy,97.71875
Train Loss,0.00293
Validation Accuracy,63.44916
Validation Loss,0.02541


### Sigmoid-BGD

In [34]:
np.random.seed(42)
input_dim=5000
hidden_dim=1024
learning_rate=0.01
num_epochs=100
activation='sigmoid'
batch_size=None
output_dim=y_train.shape[1]

In [ ]:
wandb.init(project="SMAI-A2", name="2.3-Sigmoid-BGD")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   1%|          | 1/100 [00:03<05:12,  3.15s/it]

Epoch: 0,Train Loss:0.7891820001706422,Train Accuracy:0.0,Train Hamming Accuracy:48.964869938321264
Epoch: 0,Validation Loss:0.2916091049663835,Validation Accuracy:2.1235521235521233,Validation Hamming Accuracy:97.84927784927785


Training Progress:  10%|█         | 10/100 [00:26<03:42,  2.47s/it]

Epoch: 9,Train Loss:0.0772766597315553,Train Accuracy:2.0273531777956557,Train Hamming Accuracy:98.61321176365423
Epoch: 9,Validation Loss:0.0747539416108308,Validation Accuracy:1.9305019305019304,Validation Hamming Accuracy:98.6064636064636


...

Epoch: 89,Train Loss:0.04574099975830371,Train Accuracy:23.28238133547868,Train Hamming Accuracy:98.87816215249843
Epoch: 89,Validation Loss:0.046589931537632724,Validation Accuracy:22.00772200772201,Validation Hamming Accuracy:98.86171886171886


Training Progress: 100%|██████████| 100/100 [09:17<00:00,  5.57s/it]

Epoch: 99,Train Loss:0.045030159433943534,Train Accuracy:24.29605792437651,Train Hamming Accuracy:98.89014034146778
Epoch: 99,Validation Loss:0.04594484362165954,Validation Accuracy:23.616473616473616,Validation Hamming Accuracy:98.87959387959387


Test Loss:0.04639828276732899,Test Accuracy:27.82378270950646,Test Hamming Accuracy:98.9164918479261


Epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
Train Accuracy,▁▂▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
Train Loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▁▁▁▁▁▁▁▁▁▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇████
Validation Loss,█▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,99
Train Accuracy,24.29606
Train Loss,0.04503
Validation Accuracy,23.61647
Validation Loss,0.04594


In [23]:
np.random.seed(42)
input_dim=5000
hidden_dim=512
learning_rate=0.01
num_epochs=100
activation='sigmoid'
batch_size=None
output_dim=y_train.shape[1]

In [24]:
wandb.init(project="SMAI-A2", name="2.3-Sigmoid-BGD-Hyperparam1")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   1%|          | 1/100 [00:00<01:31,  1.08it/s]

Epoch: 0,Train Loss:0.7499559578409957,Train Accuracy:0.0,Train Hamming Accuracy:51.51997854652722
Epoch: 0,Validation Loss:0.4693891636720577,Validation Accuracy:0.0,Validation Hamming Accuracy:83.76876876876877


Training Progress:  10%|█         | 10/100 [00:09<01:31,  1.01s/it]

Epoch: 9,Train Loss:0.11632326383679072,Train Accuracy:2.1238938053097343,Train Hamming Accuracy:98.60659694288013
Epoch: 9,Validation Loss:0.11000939060650931,Validation Accuracy:2.6383526383526386,Validation Hamming Accuracy:98.61933361933362


...

Training Progress:  90%|█████████ | 90/100 [01:25<00:09,  1.05it/s]

Epoch: 89,Train Loss:0.04850661365761233,Train Accuracy:21.512469831053902,Train Hamming Accuracy:98.85563600607848
Epoch: 89,Validation Loss:0.04943198787883973,Validation Accuracy:21.299871299871302,Validation Hamming Accuracy:98.85099385099385


Training Progress: 100%|██████████| 100/100 [01:34<00:00,  1.06it/s]

Epoch: 99,Train Loss:0.04747856276566484,Train Accuracy:22.992759452936447,Train Hamming Accuracy:98.8731563421829
Epoch: 99,Validation Loss:0.04847368748214111,Validation Accuracy:22.00772200772201,Validation Hamming Accuracy:98.86100386100387
Test Loss:0.0485971065178822,Test Accuracy:23.61709175223584,Test Hamming Accuracy:98.86643848220528


Epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇██
Train Accuracy,▁▁▁▂▂▁▂▂▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Train Loss,█▆▃▂▂█▆▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇████
Validation Loss,▆▄▄▃▃▂▂▂█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,99
Train Accuracy,22.99276
Train Loss,0.04748
Validation Accuracy,22.00772
Validation Loss,0.04847


In [25]:
np.random.seed(42)
input_dim=5000
hidden_dim=1024
learning_rate=0.001
num_epochs=100
activation='sigmoid'
batch_size=None
output_dim=y_train.shape[1]

In [27]:
wandb.init(project="SMAI-A2", name="2.3-Sigmoid-BGD-Hyperparam2")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   1%|          | 1/100 [00:01<02:49,  1.71s/it]

Epoch: 0,Train Loss:0.7423166923026822,Train Accuracy:0.0,Train Hamming Accuracy:51.272727272727266
Epoch: 0,Validation Loss:0.6814005648097121,Validation Accuracy:0.0,Validation Hamming Accuracy:58.33547833547833


Training Progress:  10%|█         | 10/100 [00:17<02:41,  1.79s/it]

Epoch: 9,Train Loss:0.37351986465890885,Train Accuracy:0.016090104585679808,Train Hamming Accuracy:94.95074640207383
Epoch: 9,Validation Loss:0.3538128819244488,Validation Accuracy:0.06435006435006435,Validation Hamming Accuracy:95.82296582296583


...

Epoch: 89,Train Loss:0.08288207311982168,Train Accuracy:2.091713596138375,Train Hamming Accuracy:98.61410565835345
Epoch: 89,Validation Loss:0.08312347995696216,Validation Accuracy:2.9601029601029603,Validation Hamming Accuracy:98.62076362076363


Training Progress: 100%|██████████| 100/100 [03:02<00:00,  1.83s/it]

Epoch: 99,Train Loss:0.07870746837954151,Train Accuracy:2.236524537409493,Train Hamming Accuracy:98.61625100563154
Epoch: 99,Validation Loss:0.07901138725116832,Validation Accuracy:3.0244530244530243,Validation Hamming Accuracy:98.62147862147862


Test Loss:0.07893790986144496,Test Accuracy:2.0536601523683338,Test Hamming Accuracy:98.59298516801


Epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇███
Train Accuracy,▁▁▁▁▁▁▁▂▂▂▂▂▃▃▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇█████
Train Loss,█▇▆▆▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▁▁▁▁▂▂▂▂▂▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇██████████
Validation Loss,█▇▇▆▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,99
Train Accuracy,2.23652
Train Loss,0.07871
Validation Accuracy,3.02445
Validation Loss,0.07901


### Sigmoid-SGD

In [37]:
np.random.seed(42)
input_dim=5000
hidden_dim=1024
learning_rate=0.01
num_epochs=3
activation='sigmoid'
batch_size=1
output_dim=y_train.shape[1]

In [38]:
wandb.init(project="SMAI-A2", name="2.3-Sigmoid-SGD")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:  33%|███▎      | 1/3 [03:41<07:22, 221.10s/it]

Epoch: 0,Train Loss:0.024401139035744337,Train Accuracy:61.93081255028158,Train Hamming Accuracy:99.33690891212711
Epoch: 0,Validation Loss:0.017288227181201696,Validation Accuracy:71.4929214929215,Validation Hamming Accuracy:99.48233948233948


Training Progress: 100%|██████████| 3/3 [11:12<00:00, 224.14s/it]


Test Loss:0.019565880519674388,Test Accuracy:71.18251076515403,Test Hamming Accuracy:99.47664789665453


Epoch,▁▅█
Train Accuracy,▁▇█
Train Loss,█▂▁
Validation Accuracy,▁▄█
Validation Loss,█▂▁
Epoch,2
Train Accuracy,97.05551
Train Loss,0.00218
Validation Accuracy,75.22523
Validation Loss,0.01565


In [39]:
np.random.seed(42)
input_dim=5000
hidden_dim=512
learning_rate=0.01
num_epochs=3
activation='sigmoid'
batch_size=1
output_dim=y_train.shape[1]

In [40]:
wandb.init(project="SMAI-A2", name="2.3-Sigmoid-SGD-Hyperparam1")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:  33%|███▎      | 1/3 [02:16<04:32, 136.02s/it]

Epoch: 0,Train Loss:0.023288321173470863,Train Accuracy:62.25261464199517,Train Hamming Accuracy:99.35997139536651
Epoch: 0,Validation Loss:0.01603156891082909,Validation Accuracy:72.65122265122265,Validation Hamming Accuracy:99.51665951665952


Training Progress: 100%|██████████| 3/3 [06:12<00:00, 124.14s/it]


Test Loss:0.018498521632269782,Test Accuracy:71.51374627360053,Test Hamming Accuracy:99.47812005446984


Epoch,▁▅█
Train Accuracy,▁▇█
Train Loss,█▂▁
Validation Accuracy,▁▆█
Validation Loss,█▁▂
Epoch,2
Train Accuracy,96.62108
Train Loss,0.00246
Validation Accuracy,74.71042
Validation Loss,0.01473


In [41]:
np.random.seed(42)
input_dim=5000
hidden_dim=1024
learning_rate=0.001
num_epochs=3
activation='sigmoid'
batch_size=1
output_dim=y_train.shape[1]

In [42]:
wandb.init(project="SMAI-A2", name="2.3-Sigmoid-SGD-Hyperparam2")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:  33%|███▎      | 1/3 [03:56<07:53, 236.79s/it]

Epoch: 0,Train Loss:0.04203529521793569,Train Accuracy:35.01206757843926,Train Hamming Accuracy:98.95485831768423
Epoch: 0,Validation Loss:0.03118692328922006,Validation Accuracy:47.554697554697555,Validation Hamming Accuracy:99.17703417703417


Training Progress: 100%|██████████| 3/3 [11:15<00:00, 225.06s/it]


Test Loss:0.023166214307534294,Test Accuracy:61.841669426962575,Test Hamming Accuracy:99.35151448235251


Epoch,▁▅█
Train Accuracy,▁▆█
Train Loss,█▃▁
Validation Accuracy,▁▅█
Validation Loss,█▃▁
Epoch,2
Train Accuracy,69.83105
Train Loss,0.01683
Validation Accuracy,62.67696
Validation Loss,0.0204


### Sigmoid-MBGD

In [43]:
np.random.seed(42)
input_dim=5000
hidden_dim=1024
learning_rate=0.01
num_epochs=100
activation='sigmoid'
batch_size=256
output_dim=y_train.shape[1]

In [44]:
wandb.init(project="SMAI-A2", name="2.3-Sigmoid-MBGD")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   1%|          | 1/100 [00:02<03:54,  2.37s/it]

Epoch: 0,Train Loss:0.11635941513894016,Train Accuracy:2.9379401408450705,Train Hamming Accuracy:96.61713615023474
Epoch: 0,Validation Loss:0.05684076165745996,Validation Accuracy:5.469755469755469,Validation Hamming Accuracy:98.66008866008866


Training Progress:  10%|█         | 10/100 [00:24<03:41,  2.46s/it]

Epoch: 9,Train Loss:0.03841236472696571,Train Accuracy:34.690140845070424,Train Hamming Accuracy:99.00438184663535
Epoch: 9,Validation Loss:0.03970416862631462,Validation Accuracy:33.46203346203346,Validation Hamming Accuracy:98.998998998999


...

Epoch: 89,Train Loss:0.011576864800573943,Train Accuracy:79.08296654929578,Train Hamming Accuracy:99.67810543818467
Epoch: 89,Validation Loss:0.01917213593534106,Validation Accuracy:63.51351351351351,Validation Hamming Accuracy:99.41512941512941


Training Progress: 100%|██████████| 100/100 [04:00<00:00,  2.41s/it]

Epoch: 99,Train Loss:0.010564672080506538,Train Accuracy:81.13314260563381,Train Hamming Accuracy:99.7093921165884
Epoch: 99,Validation Loss:0.01850740588472731,Validation Accuracy:64.41441441441441,Validation Hamming Accuracy:99.42871442871443


Test Loss:0.021517138756212445,Test Accuracy:63.6303411725737,Test Hamming Accuracy:99.3802215597512


Epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇█
Train Accuracy,▁▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████
Train Loss,█▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▁▄▃▄▄▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████████
Validation Loss,█▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,99
Train Accuracy,81.13314
Train Loss,0.01056
Validation Accuracy,64.41441
Validation Loss,0.01851


In [45]:
np.random.seed(42)
input_dim=5000
hidden_dim=512
learning_rate=0.01
num_epochs=100
activation='sigmoid'
batch_size=256
output_dim=y_train.shape[1]

In [46]:
wandb.init(project="SMAI-A2", name="2.3-Sigmoid-MBGD-Hyperparam1")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   1%|          | 1/100 [00:01<01:56,  1.18s/it]

Epoch: 0,Train Loss:0.1585590482961887,Train Accuracy:2.450264084507042,Train Hamming Accuracy:96.0897055946792
Epoch: 0,Validation Loss:0.06948695582264684,Validation Accuracy:5.341055341055341,Validation Hamming Accuracy:98.65293865293864


Training Progress:  10%|█         | 10/100 [00:12<01:51,  1.24s/it]

Epoch: 9,Train Loss:0.039305269871086644,Train Accuracy:33.71764964788733,Train Hamming Accuracy:98.9998728482003
Epoch: 9,Validation Loss:0.04073399595353394,Validation Accuracy:31.467181467181465,Validation Hamming Accuracy:98.97039897039896


...

Epoch: 89,Train Loss:0.012536079166320855,Train Accuracy:76.77332746478874,Train Hamming Accuracy:99.63903315727701
Epoch: 89,Validation Loss:0.019717688083113895,Validation Accuracy:63.06306306306306,Validation Hamming Accuracy:99.41012441012441


Training Progress: 100%|██████████| 100/100 [02:05<00:00,  1.26s/it]

Epoch: 99,Train Loss:0.011304344730748513,Train Accuracy:79.25528169014085,Train Hamming Accuracy:99.68304479655713
Epoch: 99,Validation Loss:0.019006667036199517,Validation Accuracy:63.899613899613904,Validation Hamming Accuracy:99.42728442728442


Test Loss:0.021600161856075207,Test Accuracy:63.03411725736999,Test Hamming Accuracy:99.37212469176696


Epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇████
Train Accuracy,▁▁▂▂▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇███████
Train Loss,█▆▆▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▂▃▃▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
Validation Loss,██▇▇▇▆▆▆▅▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
Epoch,99
Train Accuracy,79.25528
Train Loss,0.0113
Validation Accuracy,63.89961
Validation Loss,0.01901


In [47]:
np.random.seed(42)
input_dim=5000
hidden_dim=1024
learning_rate=0.001
num_epochs=100
activation='sigmoid'
batch_size=256
output_dim=y_train.shape[1]

In [48]:
wandb.init(project="SMAI-A2", name="2.3-Sigmoid-MBGD-Hyperparam2")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   1%|          | 1/100 [00:02<03:57,  2.40s/it]

Epoch: 0,Train Loss:0.3831871651682758,Train Accuracy:1.1283010563380282,Train Hamming Accuracy:88.57452807120502
Epoch: 0,Validation Loss:0.1943124654994644,Validation Accuracy:2.0592020592020592,Validation Hamming Accuracy:98.55069355069355


Training Progress:  10%|█         | 10/100 [00:23<03:28,  2.31s/it]

Epoch: 9,Train Loss:0.05782209943893465,Train Accuracy:5.006602112676056,Train Hamming Accuracy:98.6514377934272
Epoch: 9,Validation Loss:0.05765396668020819,Validation Accuracy:4.633204633204633,Validation Hamming Accuracy:98.65579865579865


...

Training Progress:  90%|█████████ | 90/100 [03:31<00:23,  2.38s/it]

Epoch: 89,Train Loss:0.03877874550489133,Train Accuracy:33.114436619718305,Train Hamming Accuracy:98.99139280125196
Epoch: 89,Validation Loss:0.04053169493740226,Validation Accuracy:31.402831402831406,Validation Hamming Accuracy:98.97182897182897


Training Progress: 100%|██████████| 100/100 [03:56<00:00,  2.36s/it]


Epoch: 99,Train Loss:0.03786996125942644,Train Accuracy:34.895686619718305,Train Hamming Accuracy:99.01017458920185
Epoch: 99,Validation Loss:0.039696923165221755,Validation Accuracy:33.14028314028314,Validation Hamming Accuracy:98.992563992564
Test Loss:0.04043700442969204,Test Accuracy:36.999006293474665,Test Hamming Accuracy:99.02543152625961


Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇███
Train Accuracy,▁▁▁▁▂▃▄▄▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████
Train Loss,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▁▁▁▁▂▂▃▃▄▅▆▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█████
Validation Loss,█▇▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,99
Train Accuracy,34.89569
Train Loss,0.03787
Validation Accuracy,33.14028
Validation Loss,0.0397


### Tanh-BGD

In [49]:
np.random.seed(42)
input_dim=5000
hidden_dim=1024
learning_rate=0.01
num_epochs=100
activation='tanh'
batch_size=None
output_dim=y_train.shape[1]

In [50]:
wandb.init(project="SMAI-A2", name="2.3-Tanh-BGD")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   1%|          | 1/100 [00:01<02:48,  1.70s/it]

Epoch: 0,Train Loss:0.7981931924572796,Train Accuracy:0.0,Train Hamming Accuracy:50.09064092249933
Epoch: 0,Validation Loss:0.8026428389097112,Validation Accuracy:0.0,Validation Hamming Accuracy:50.51122551122551


Training Progress:  10%|█         | 10/100 [00:17<02:33,  1.71s/it]

Epoch: 9,Train Loss:0.7433229948590461,Train Accuracy:0.0,Train Hamming Accuracy:55.20282470724949
Epoch: 9,Validation Loss:0.7569317391121562,Validation Accuracy:0.0,Validation Hamming Accuracy:54.88130988130988


...

Epoch: 89,Train Loss:0.2133313402038181,Train Accuracy:32.30893000804505,Train Hamming Accuracy:97.31331009207116
Epoch: 89,Validation Loss:0.32170394438958894,Validation Accuracy:18.597168597168597,Validation Hamming Accuracy:89.61461461461462


Training Progress: 100%|██████████| 100/100 [02:58<00:00,  1.78s/it]

Epoch: 99,Train Loss:0.17054898261378768,Train Accuracy:42.799678197908285,Train Hamming Accuracy:98.46071332796996
Epoch: 99,Validation Loss:0.2811714975613386,Validation Accuracy:23.680823680823682,Validation Hamming Accuracy:91.72458172458172


Test Loss:0.2758656310600811,Test Accuracy:27.2606823451474,Test Hamming Accuracy:91.91196496264399


Epoch,▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇███
Train Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▇███
Train Loss,█████▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁
Validation Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▆▇███
Validation Loss,█████▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▂▂▁▁▁▁
Epoch,99
Train Accuracy,42.79968
Train Loss,0.17055
Validation Accuracy,23.68082
Validation Loss,0.28117


In [51]:
np.random.seed(42)
input_dim=5000
hidden_dim=512
learning_rate=0.01
num_epochs=100
activation='tanh'
batch_size=None
output_dim=y_train.shape[1]

In [52]:
wandb.init(project="SMAI-A2", name="2.3-Tanh-BGD-Hyperparam1")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   1%|          | 1/100 [00:00<01:28,  1.11it/s]

Epoch: 0,Train Loss:0.7998356122094562,Train Accuracy:0.0,Train Hamming Accuracy:49.82998122821132
Epoch: 0,Validation Loss:0.8019334045621197,Validation Accuracy:0.0,Validation Hamming Accuracy:50.25454025454026


Training Progress:  10%|█         | 10/100 [00:09<01:28,  1.02it/s]

Epoch: 9,Train Loss:0.7457996534006813,Train Accuracy:0.0,Train Hamming Accuracy:54.86564762670957
Epoch: 9,Validation Loss:0.7566415312843794,Validation Accuracy:0.0,Validation Hamming Accuracy:54.53381953381954


...

Training Progress:  90%|█████████ | 90/100 [01:22<00:09,  1.10it/s]

Epoch: 89,Train Loss:0.22424996483916348,Train Accuracy:30.15285599356396,Train Hamming Accuracy:96.76946455707518
Epoch: 89,Validation Loss:0.3261926014959074,Validation Accuracy:17.696267696267697,Validation Hamming Accuracy:89.28356928356929


Training Progress: 100%|██████████| 100/100 [01:31<00:00,  1.09it/s]

Epoch: 99,Train Loss:0.1807607139617072,Train Accuracy:39.469026548672566,Train Hamming Accuracy:98.10136765888979
Epoch: 99,Validation Loss:0.2853957059954624,Validation Accuracy:22.393822393822393,Validation Hamming Accuracy:91.58229658229658
Test Loss:0.282250225074345,Test Accuracy:26.00198741305068,Test Hamming Accuracy:91.6996061977844


Epoch,▁▁▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇█████
Train Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▅▅▅▅▆▇▇██
Train Loss,█████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▁▁▁▁▁▁
Validation Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▆▆▆▇▇▇██
Validation Loss,███▇▇▇▇▇▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
Epoch,99
Train Accuracy,39.46903
Train Loss,0.18076
Validation Accuracy,22.39382
Validation Loss,0.2854


In [53]:
np.random.seed(42)
input_dim=5000
hidden_dim=1024
learning_rate=0.001
num_epochs=100
activation='tanh'
batch_size=None
output_dim=y_train.shape[1]

In [54]:
wandb.init(project="SMAI-A2", name="2.3-Tanh-BGD-Hyperparam2")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   1%|          | 1/100 [00:01<02:49,  1.71s/it]

Epoch: 0,Train Loss:0.7981931924572796,Train Accuracy:0.0,Train Hamming Accuracy:50.09064092249933
Epoch: 0,Validation Loss:0.8076656634584923,Validation Accuracy:0.0,Validation Hamming Accuracy:50.03003003003003


Training Progress:  10%|█         | 10/100 [00:17<02:41,  1.79s/it]

Epoch: 9,Train Loss:0.7923195401300261,Train Accuracy:0.0,Train Hamming Accuracy:50.62232948958613
Epoch: 9,Validation Loss:0.8026748100769147,Validation Accuracy:0.0,Validation Hamming Accuracy:50.509080509080505


...

Epoch: 89,Train Loss:0.7439492067001747,Train Accuracy:0.0,Train Hamming Accuracy:55.144006436041835
Epoch: 89,Validation Loss:0.7618237503137242,Validation Accuracy:0.0,Validation Hamming Accuracy:54.383669383669385


Training Progress: 100%|██████████| 100/100 [02:56<00:00,  1.77s/it]

Epoch: 99,Train Loss:0.7381791304976615,Train Accuracy:0.0,Train Hamming Accuracy:55.683918834361315
Epoch: 99,Validation Loss:0.7569910633407433,Validation Accuracy:0.0,Validation Hamming Accuracy:54.869869869869866


Test Loss:0.7470200515286103,Test Accuracy:0.0,Test Hamming Accuracy:55.45912921865224


Epoch,▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
Train Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Loss,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
Validation Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Loss,████▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁
Epoch,99
Train Accuracy,0
Train Loss,0.73818
Validation Accuracy,0
Validation Loss,0.75699


### Tanh-SGD

In [8]:
np.random.seed(42)
input_dim=5000
hidden_dim=1024
learning_rate=0.01
num_epochs=3
activation='tanh'
batch_size=1
output_dim=y_train.shape[1]

In [9]:
wandb.init(project="SMAI-A2", name="2.3-Tanh-SGD")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:  33%|███▎      | 1/3 [03:45<07:31, 225.57s/it]

Epoch: 0,Train Loss:0.1445257757137157,Train Accuracy:50.74818986323411,Train Hamming Accuracy:94.96397604361879
Epoch: 0,Validation Loss:0.02952928795226433,Validation Accuracy:63.25611325611325,Validation Hamming Accuracy:99.2935792935793


Training Progress: 100%|██████████| 3/3 [10:58<00:00, 219.52s/it]


Test Loss:0.02820592314086401,Test Accuracy:68.00264988406757,Test Hamming Accuracy:99.34746604836039


Epoch,▁▅█
Train Accuracy,▁▆█
Train Loss,█▁▁
Validation Accuracy,▁▅█
Validation Loss,█▃▁
Epoch,2
Train Accuracy,96.73371
Train Loss,0.00287
Validation Accuracy,70.14157
Validation Loss,0.02403


In [9]:
np.random.seed(42)
input_dim=5000
hidden_dim=512
learning_rate=0.01
num_epochs=1
activation='tanh'
batch_size=1
output_dim=y_train.shape[1]

In [10]:
wandb.init(project="SMAI-A2", name="2.3-Tanh-SGD-Hyperparam1")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress: 100%|██████████| 1/1 [01:52<00:00, 112.04s/it]

Epoch: 0,Train Loss:0.09926349115529617,Train Accuracy:56.379726468222046,Train Hamming Accuracy:96.714579422541
Epoch: 0,Validation Loss:0.026054383875355934,Validation Accuracy:67.11711711711712,Validation Hamming Accuracy:99.4036894036894
Test Loss:0.027713203119533927,Test Accuracy:66.97581980788341,Test Hamming Accuracy:99.36697213941335


Epoch,▁
Train Accuracy,▁
Train Loss,▁
Validation Accuracy,▁
Validation Loss,▁
Epoch,0
Train Accuracy,56.37973
Train Loss,0.09926
Validation Accuracy,67.11712
Validation Loss,0.02605


In [11]:
np.random.seed(42)
input_dim=5000
hidden_dim=1024
learning_rate=0.001
num_epochs=1
activation='tanh'
batch_size=1
output_dim=y_train.shape[1]

In [12]:
wandb.init(project="SMAI-A2", name="2.3-Tanh-SGD-Hyperparam2")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress: 100%|██████████| 1/1 [03:32<00:00, 212.82s/it]

Epoch: 0,Train Loss:0.34666748559311117,Train Accuracy:32.08366854384553,Train Hamming Accuracy:85.78939840886538
Epoch: 0,Validation Loss:0.05509684225335916,Validation Accuracy:61.518661518661524,Validation Hamming Accuracy:99.27570427570429


Test Loss:0.055559012263962974,Test Accuracy:61.77542232527327,Test Hamming Accuracy:99.2127636082588


Epoch,▁
Train Accuracy,▁
Train Loss,▁
Validation Accuracy,▁
Validation Loss,▁
Epoch,0
Train Accuracy,32.08367
Train Loss,0.34667
Validation Accuracy,61.51866
Validation Loss,0.0551


### Tanh-MBGD

In [10]:
np.random.seed(42)
input_dim=5000
hidden_dim=1024
learning_rate=0.01
num_epochs=100
activation='tanh'
batch_size=256
output_dim=y_train.shape[1]

In [11]:
wandb.init(project="SMAI-A2", name="2.3-Tanh-MBGD")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   1%|          | 1/100 [00:02<04:17,  2.60s/it]

Epoch: 0,Train Loss:0.7497257892861667,Train Accuracy:0.046875,Train Hamming Accuracy:54.95882726917059
Epoch: 0,Validation Loss:0.6902901908824873,Validation Accuracy:0.2574002574002574,Validation Hamming Accuracy:60.82439582439583


Training Progress:  10%|█         | 10/100 [00:25<03:41,  2.46s/it]

Epoch: 9,Train Loss:0.03711850342285755,Train Accuracy:64.63270246478874,Train Hamming Accuracy:99.4453834115806
Epoch: 9,Validation Loss:0.10743453641653455,Validation Accuracy:50.51480051480052,Validation Hamming Accuracy:97.78778778778778


...

Epoch: 89,Train Loss:0.0029057000669519455,Train Accuracy:98.63732394366198,Train Hamming Accuracy:99.98277582159625
Epoch: 89,Validation Loss:0.03995337732371485,Validation Accuracy:69.56241956241956,Validation Hamming Accuracy:98.98755898755898


Training Progress: 100%|██████████| 100/100 [04:02<00:00,  2.42s/it]

Epoch: 99,Train Loss:0.002626435748669192,Train Accuracy:98.74053697183099,Train Hamming Accuracy:99.9836438771518
Epoch: 99,Validation Loss:0.038765515608697815,Validation Accuracy:69.94851994851994,Validation Hamming Accuracy:99.006149006149


Test Loss:0.04240147655453799,Test Accuracy:67.47267307055317,Test Hamming Accuracy:98.89588163851165


Epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
Train Accuracy,▁▂▅▆▆▆▆▇▇▇▇▇▇███████████████████████████
Train Loss,█▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▁▅▆▆▆▇▇▇▇▇▇▇▇▇▇████████████████████████
Validation Loss,█▅▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,99
Train Accuracy,98.74054
Train Loss,0.00263
Validation Accuracy,69.94852
Validation Loss,0.03877


In [12]:
np.random.seed(42)
input_dim=5000
hidden_dim=512
learning_rate=0.01
num_epochs=100
activation='tanh'
batch_size=256
output_dim=y_train.shape[1]

In [13]:
wandb.init(project="SMAI-A2", name="2.3-Tanh-MBGD-Hyperparam1")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   1%|          | 1/100 [00:01<02:02,  1.24s/it]

Epoch: 0,Train Loss:0.7500627906668774,Train Accuracy:0.0,Train Hamming Accuracy:54.8043989632238
Epoch: 0,Validation Loss:0.691591261396555,Validation Accuracy:0.0,Validation Hamming Accuracy:60.52838552838553


Training Progress:  10%|█         | 10/100 [00:13<01:59,  1.33s/it]

Epoch: 9,Train Loss:0.04114693762690697,Train Accuracy:60.44718309859155,Train Hamming Accuracy:99.37714935446009
Epoch: 9,Validation Loss:0.10773486251539995,Validation Accuracy:48.51994851994852,Validation Hamming Accuracy:97.83283283283282


...

Epoch: 89,Train Loss:0.0035168070116096983,Train Accuracy:98.38116197183099,Train Hamming Accuracy:99.97940874413143
Epoch: 89,Validation Loss:0.03809061943935453,Validation Accuracy:70.07722007722008,Validation Hamming Accuracy:99.02259402259402


Training Progress: 100%|██████████| 100/100 [02:15<00:00,  1.36s/it]

Epoch: 99,Train Loss:0.0031230921070049223,Train Accuracy:98.515625,Train Hamming Accuracy:99.98107638888888
Epoch: 99,Validation Loss:0.036867135471129214,Validation Accuracy:70.20592020592021,Validation Hamming Accuracy:99.04547404547405


Test Loss:0.04072350989942784,Test Accuracy:67.47267307055317,Test Hamming Accuracy:98.92458871591035


Epoch,▁▁▁▁▂▂▂▂▂▂▃▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇███
Train Accuracy,▁▁▁▄▅▆▆▆▆▇▇▇▇▇▇▇████████████████████████
Train Loss,█▅▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▂▄▄▅▆▆▇▇▇▇█████████████████████████████
Validation Loss,█▇▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,99
Train Accuracy,98.51562
Train Loss,0.00312
Validation Accuracy,70.20592
Validation Loss,0.03687


In [14]:
np.random.seed(42)
input_dim=5000
hidden_dim=1024
learning_rate=0.001
num_epochs=100
activation='tanh'
batch_size=256
output_dim=y_train.shape[1]

In [15]:
wandb.init(project="SMAI-A2", name="2.3-Tanh-MBGD-Hyperparam2")
print("Initialising model")
model = MLP(input_dim, hidden_dim, output_dim,activation)   
model.train(x_train,y_train,num_epochs,learning_rate,x_val,y_val,batch_size)
y_pred_test=model.predict(x_test)
test_loss=model.binary_cross_entropy(y_test,y_pred_test)
test_accuracy,test_hamming_accuracy=model.calculate_accuracy(y_test,y_pred_test)
print(f"Test Loss:{test_loss},Test Accuracy:{test_accuracy},Test Hamming Accuracy:{test_hamming_accuracy}")

wandb.finish()

Initialising model


Training Progress:   1%|          | 1/100 [00:02<03:42,  2.25s/it]

Epoch: 0,Train Loss:0.7930537787940011,Train Accuracy:0.0,Train Hamming Accuracy:50.65240610328638
Epoch: 0,Validation Loss:0.7944673254510947,Validation Accuracy:0.0,Validation Hamming Accuracy:51.19548119548119


Training Progress:  10%|█         | 10/100 [00:24<03:47,  2.52s/it]

Epoch: 9,Train Loss:0.6617990528185971,Train Accuracy:0.359375,Train Hamming Accuracy:62.87172095070422
Epoch: 9,Validation Loss:0.6863224140182728,Validation Accuracy:0.32175032175032175,Validation Hamming Accuracy:61.194766194766196


...

Epoch: 89,Train Loss:0.03861264823431397,Train Accuracy:64.10387323943661,Train Hamming Accuracy:99.44124853286385
Epoch: 89,Validation Loss:0.11941983675469532,Validation Accuracy:48.58429858429859,Validation Hamming Accuracy:97.45245245245245


Training Progress: 100%|██████████| 100/100 [04:06<00:00,  2.46s/it]

Epoch: 99,Train Loss:0.03361874512145268,Train Accuracy:65.66351232394366,Train Hamming Accuracy:99.47306093505476
Epoch: 99,Validation Loss:0.11042789968308503,Validation Accuracy:49.67824967824968,Validation Hamming Accuracy:97.66409266409266


Test Loss:0.11029588802061663,Test Accuracy:52.401457436237166,Test Hamming Accuracy:97.64086710095322


Epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇█████
Train Accuracy,▁▁▁▁▁▁▂▂▂▂▂▃▃▄▅▆▇▇▇▇▇▇▇▇▇▇██████████████
Train Loss,██▇▆▆▅▅▅▄▄▄▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▁▁▁▁▁▁▁▂▂▂▂▃▃▄▄▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇███████
Validation Loss,█▇▇▇▇▇▆▆▆▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,99
Train Accuracy,65.66351
Train Loss,0.03362
Validation Accuracy,49.67825
Validation Loss,0.11043


#### Results

| Activation | Optimizer | Input Dim | Hidden Dim | Learning Rate | Test Loss | Test Accuracy (%) | Hamming Accuracy (%) |
|------------|-----------|------------|------------|---------------|-----------|------------------|--------------------|
| **ReLU**  | **BGD**  | 5000       | 1024       | 0.01          | 0.0471    | 48.23           | 99.12             |
|           |          | 5000       | 512        | 0.01          | 0.0483    | 45.21           | 99.08             |
|           |          | 5000       | 512        | 0.001         | 0.1436    | 11.96           | 98.20             |
| **ReLU**  | **SGD**  | 5000       | 1024       | 0.01          | 0.0471    | 48.23           | 99.12             |
|           |          | 5000       | 512        | 0.01          | 0.0483    | 45.21           | 99.08             |
|           |          | 5000       | 512        | 0.001         | 0.1030    | 18.25           | 98.63             |
| **ReLU**  | **MBGD** | 5000       | 1024       | 0.01          | 0.0328    | 61.71           | 99.35             |
|           |          | 5000       | 512        | 0.01          | 0.0310    | 62.47           | 99.36             |
|           |          | 5000       | 512        | 0.001         | 0.0331    | 61.11           | 99.35             |
| **Tanh**  | **BGD**  | 5000       | 1024       | 0.01          | 0.2759    | 27.26           | 91.91             |
|           |          | 5000       | 512        | 0.01          | 0.2823    | 26.00           | 91.70             |
|           |          | 5000       | 512        | 0.001         | 0.7470    | 0.00            | 55.46             |
| **Tanh**  | **SGD**  | 5000       | 1024       | 0.01          | 0.0282    | 68.00           | 99.35             |
|           |          | 5000       | 512        | 0.01          | 0.028     | 66.98           | 99.36             |
|           |          |  5000       | 1024       | 0.01          |0.056    | 61/78            | 99.12            |
| **Tanh**  | **MBGD** | 5000       | 1024       | 0.01          | 0.0424    | 67.47           | 98.90             |
|           |          | 5000       | 512        | 0.01          | 0.0407    | 67.47           | 98.92             |
|           |          | 5000       | 512        | 0.001         | 0.1103    | 52.40           | 97.64             |
| **Sigmoid** | **BGD** | 5000     | 1024       | 0.01          | 0.0464    | 27.82           | 98.92             |
|           |          | 5000       | 512        | 0.01          | 0.0486    | 23.62           | 98.87             |
|           |          | 5000       | 512        | 0.001         | 0.0789    | 2.05            | 98.59             |
| **Sigmoid** | **SGD** | 5000     | 1024       | 0.01          | 0.0196    | 71.18           | 99.48             |
|           |          | 5000       | 512        | 0.01          | 0.0185    | 71.51           | 99.48             |
|           |          | 5000       | 512        | 0.001         | 0.0232    | 61.84           | 99.35             |
| **Sigmoid** | **MBGD** | 5000    | 1024       | 0.01          | 0.0215    | 63.63           | 99.38             |
|           |          | 5000       | 512        | 0.01          | 0.0216    | 63.03           | 99.37             |
|           |          | 5000       | 512        | 0.001         | 0.0404    | 37.00           | 99.03             |

#### Best Configuration
 
1. Effect of Activation Functions
    - ReLU performs consistently well across different optimizers and configurations, with test accuracies ranging between 45% to 62%.
    - Tanh has mixed results—it performs well in some cases but generally underperforms compared to ReLU.
    - Sigmoid performs poorly with BGD, with test accuracies as low as 2.05% for small learning rates, but improves significantly when using SGD and MBGD (up to 71.51% accuracy).
2. Effect of Optimizers
    - Batch Gradient Descent (BGD) performs poorly in most cases, especially with Tanh and Sigmoid, suggesting that it struggles with gradient updates due to possible vanishing gradients.
    - Stochastic Gradient Descent (SGD) yields the best results in most cases, with the highest test accuracy of 71.51% 
    - Mini-Batch Gradient Descent (MBGD) gives results comparable to SGD, but slightly lower in some cases.

Minibatch with sigmoid will give the best combination.

![Alt Text](Graphs\2.3_BGD_train_loss.png)
![Alt Text](Graphs\2.3_BGD_train_acc.png)
![Alt Text](Graphs\2.3_BGD_val_loss.png)
![Alt Text](Graphs\2.3._BGD_val_acc.png)

![Alt Text](Graphs\2.3_SGD_train_loss.png)
![Alt Text](Graphs\2.3_SGD_train_acc.png)
![Alt Text](Graphs\2.3_SGD_val_loss.png)
![Alt Text](Graphs\2.3_SGD_val_acc.png)

![Alt Text](Graphs\2.3_MBGD_train_loss.png)
![Alt Text](Graphs\2.3_MBGD_train_acc.png)
![Alt Text](Graphs\2.3_MBGD_val_loss.png)
![Alt Text](Graphs\2.3._MBGD_val_acc.png)